**In order to segment and cluster the neighborhoods in Toronto, we will need the below libraries installed. In order to scrape the below Wikipedia page, we will be using Beautiful Soup library.**
('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M') 


In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import urllib.request


**We will be requesting the Wiki page and reading the page in a row format. In order to make better sense of the page, we will be using "prettify" function.**

In [2]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = urllib.request.urlopen(url)

soup = BeautifulSoup(page, "lxml")

#print(soup.prettify())


In [3]:
table=soup.find('table',{'class':'wikitable sortable'})

<Header>

**Because we do not need the entire page, we will only scrape those with "tr"**

In [4]:
links = table.find_all('tr')
#links

**We need to begin appending the table in the URL into our data frame so we first begin with creating an empty list**

In [5]:
Data=[]

for link in links:
    Data.append([t.text.strip()for t in link.find_all('td')])




In [6]:
    
df=pd.DataFrame(Data,columns=['PostalCode','Borough','Neighborhood'])
df=df[df['Borough']!='Not assigned']
df = df[~df['PostalCode'].isnull()] 


df

,PostalCode,Borough,Neighborhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,"Regent Park, Harbourfront"
6,M6A,North York,"Lawrence Manor, Lawrence Heights"
7,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
161,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
166,M4Y,Downtown Toronto,Church and Wellesley
169,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
170,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


**More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.**

In [7]:
df2=df.groupby('PostalCode').agg(lambda x: ','.join(x))

df3=df.groupby('PostalCode')



**If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.**


In [8]:
 df['Neighborhood'].replace('Not Assigned',df['Borough'],inplace=True)
df                            


,PostalCode,Borough,Neighborhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,"Regent Park, Harbourfront"
6,M6A,North York,"Lawrence Manor, Lawrence Heights"
7,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
161,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
166,M4Y,Downtown Toronto,Church and Wellesley
169,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
170,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [9]:
df.shape                           


(103, 3)

**Using Geocoder, we will then extract Longitude and Latitude of each PostalCode**

In [68]:
 pip install geocoder


     |████████████████████████████████| 102kB 1.3MB/s ta 0:00:011
Note: you may need to restart the kernel to use updated packages.


In [10]:
import geocoder # import geocoder


In [15]:
post=pd.read_csv('Geospatial_Coordinates.csv')
post

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [18]:
post.rename(columns={'Postal Code':'PostalCode'}, 
                 inplace=True)
post

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [20]:
df_merge=df.merge(post, on='PostalCode', how='left')
df_merge

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509
